In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
#Muat dataset dan cek tipe data
df = pd.read_csv('https://drive.google.com/uc?id=1GTNiDZzzVbbyn9aObjsXTeGqDBBwQTpo')
print(df.head())
print(df.info())

     place_id                                          name  \
0  timur_0001                                Pantai Natsepa   
1  timur_0002                                  Pantai Liang   
2  timur_0004  Pantai Ngurbloat Desa Ngilngof - Kei Islands   
3  timur_0005                       Pantai Pintu Kota Ambon   
4  timur_0006                              Pantai Kota Jawa   

                                         description  reviews  rating  \
0  Jalur pasir putih dengan pemandian, banana boa...     2217     4.4   
1                                                  -     1246     4.5   
2  Hamparan pasir putih panjang berlatar pohon ke...     1070     4.7   
3  Tempat berenang tertutup di samping pantai ber...     1021     4.4   
4                                                  -      660     4.4   

                                      featured_image  \
0  https://lh3.ggpht.com/p/AF1QipMYYpqAHoy1IcbdyU...   
1  https://lh3.ggpht.com/p/AF1QipOQRzqn26_fRZauPI...   
2  https://lh3

In [ ]:
df.isna().sum()

,0
place_id,0
name,0
description,0
reviews,0
rating,0
featured_image,1
address,11
review_keywords,0
link,0
coordinates,0


In [ ]:
# --- Pra-pemrosesan Kolom Teks ---
df['description'] = df['description'].fillna('')
df['review_keywords'] = df['review_keywords'].fillna('')

df['combined_features'] = df['description'] + ' ' + df['review_keywords']

print("Contoh 'combined_features' (gabungan deskripsi dan keywords):")
print(df['combined_features'].head())
print("\n")

# --- Mengatur Stop Words Bahasa Indonesia dengan Sastrawi ---
factory = StopWordRemoverFactory()
stopword_sastrawi = factory.get_stop_words()

print(f"Jumlah Stop Words dari Sastrawi: {len(stopword_sastrawi)}")
print(f"Beberapa contoh Stop Words: {stopword_sastrawi[:10]}...")

# --- Inisialisasi dan Terapkan TfidfVectorizer ---
tfidf_vectorizer = TfidfVectorizer(stop_words=stopword_sastrawi, min_df=2, max_df=0.85)

tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

print(f"\nBentuk Matriks TF-IDF (Dokumen x Fitur): {tfidf_matrix.shape}")
print(f"Jumlah fitur (kata unik) yang digunakan untuk kemiripan: {len(tfidf_vectorizer.get_feature_names_out())}")



Contoh 'combined_features' (gabungan deskripsi dan keywords):
0    Jalur pasir putih dengan pemandian, banana boa...
1    - tempat wisata, pemandangan, rujak, banana bo...
2    Hamparan pasir putih panjang berlatar pohon ke...
3    Tempat berenang tertutup di samping pantai ber...
4    - santai, sore, pemandangan, snack, pinggir pa...
Name: combined_features, dtype: object


Jumlah Stop Words dari Sastrawi: 126
Beberapa contoh Stop Words: ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua']...

Bentuk Matriks TF-IDF (Dokumen x Fitur): (2392, 1261)
Jumlah fitur (kata unik) yang digunakan untuk kemiripan: 1261


In [ ]:
# Sel 4: Menghitung Kemiripan dan Membuat Fungsi Rekomendasi (Direvisi)

# --- Menghitung Kemiripan Kosinus (Cosine Similarity) ---
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print(f"Bentuk Matriks Kemiripan Kosinus: {cosine_sim.shape}")
print("\n")

# --- Membuat Mapping Nama Tempat ke Indeks DataFrame ---
# Menggunakan .reset_index() dan kemudian set_index('name') untuk memastikan bahwa
# setiap 'name' yang unik memiliki satu entri indeks.
# Kemudian, saat mengambil indeks, kita bisa menggunakan .iloc[0] jika
# hasilnya masih berupa Series (misalnya, karena duplikasi nama di data asli).
indices = pd.Series(df.index, index=df['name']).drop_duplicates()
print("Mapping nama tempat ke indeks dibuat.")


# --- Fungsi Rekomendasi Berbasis Konten ---
def get_content_based_recommendations(place_name, cosine_sim_matrix=cosine_sim, df_data=df, indices_map=indices, num_recommendations=10):
    """
    Memberikan rekomendasi tempat wisata berdasarkan kemiripan konten (Content-Based Filtering).
    """
    if place_name not in indices_map.index:
        print(f"Maaf, tempat '{place_name}' tidak ditemukan dalam daftar.")
        matching_places = [name for name in indices_map.index if place_name.lower() in name.lower()]
        if matching_places:
            print(f"Mungkin maksud Anda: {', '.join(matching_places[:5])}")
        return pd.DataFrame()

    # REVISI PENTING DI SINI: Pastikan idx adalah integer tunggal
    # Jika indices_map[place_name] mengembalikan Series (karena nama duplikat), ambil indeks pertama.
    idx_raw = indices_map[place_name]
    if isinstance(idx_raw, pd.Series): # Jika hasilnya Series (nama duplikat)
        idx = idx_raw.iloc[0] # Ambil indeks pertama
        print(f"Peringatan: Ditemukan duplikasi nama '{place_name}'. Menggunakan indeks pertama: {idx}")
    else: # Jika sudah integer tunggal
        idx = idx_raw


    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [s for s in sim_scores if s[0] != idx]
    top_similar_places_indices = [s[0] for s in sim_scores[0:num_recommendations]]
    return df_data.iloc[top_similar_places_indices]

# --- Fungsi Rekomendasi Berdasarkan Popularitas/Rating (Tidak ada perubahan) ---
def get_popular_recommendations(df_data=df, num_recommendations=10):
    df_data['rating'] = pd.to_numeric(df_data['rating'], errors='coerce')
    df_data['reviews'] = pd.to_numeric(df_data['reviews'], errors='coerce')
    df_data_filtered = df_data.dropna(subset=['rating', 'reviews']).copy()
    popular_places = df_data_filtered.sort_values(by=['rating', 'reviews'], ascending=[False, False])
    return popular_places.head(num_recommendations)

print("Fungsi rekomendasi 'get_content_based_recommendations' dan 'get_popular_recommendations' berhasil didefinisikan.")

Bentuk Matriks Kemiripan Kosinus: (2392, 2392)


Mapping nama tempat ke indeks dibuat.
Fungsi rekomendasi 'get_content_based_recommendations' dan 'get_popular_recommendations' berhasil didefinisikan.


In [ ]:
import pickle # Pustaka untuk serialisasi objek Python

# --- Artefak yang perlu disimpan ---
# 1. df: DataFrame yang sudah bersih dan memiliki 'combined_features'.
#    (Meskipun bisa dimuat ulang dari CSV, menyimpannya memastikan konsistensi setelah pra-pemrosesan).
# 2. tfidf_vectorizer: Objek TfidfVectorizer yang sudah dilatih (fit), digunakan untuk memproses input baru.
# 3. cosine_sim: Matriks kemiripan yang sudah dihitung.
# 4. indices: Mapping nama ke indeks untuk pencarian cepat.

print("Menyimpan artefak model...")

# Menyimpan DataFrame
try:
    df.to_pickle('df_places.pkl') # Metode to_pickle() bawaan Pandas
    print("DataFrame 'df_places.pkl' berhasil disimpan.")
except Exception as e:
    print(f"Gagal menyimpan DataFrame: {e}")

# Menyimpan TfidfVectorizer menggunakan pickle
try:
    with open('tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    print("TfidfVectorizer 'tfidf_vectorizer.pkl' berhasil disimpan.")
except Exception as e:
    print(f"Gagal menyimpan TfidfVectorizer: {e}")

# Menyimpan Cosine Similarity Matrix menggunakan numpy.save
# np.save adalah cara yang efisien untuk menyimpan array NumPy.
try:
    np.save('cosine_sim.npy', cosine_sim)
    print("Matriks Cosine Similarity 'cosine_sim.npy' berhasil disimpan.")
except Exception as e:
    print(f"Gagal menyimpan Cosine Similarity Matrix: {e}")

# Menyimpan Mapping Indices menggunakan pickle
try:
    with open('indices_map.pkl', 'wb') as f:
        pickle.dump(indices, f)
    print("Mapping Indices 'indices_map.pkl' berhasil disimpan.")
except Exception as e:
    print(f"Gagal menyimpan Mapping Indices: {e}")

print("\nProses 'Build' (penyimpanan artefak model) selesai.")
print("File-file berikut telah dibuat di direktori kerja Anda:")
print("- df_places.pkl")
print("- tfidf_vectorizer.pkl")
print("- cosine_sim.npy")
print("- indices_map.pkl")


Menyimpan artefak model...
DataFrame 'df_places.pkl' berhasil disimpan.
TfidfVectorizer 'tfidf_vectorizer.pkl' berhasil disimpan.
Matriks Cosine Similarity 'cosine_sim.npy' berhasil disimpan.
Mapping Indices 'indices_map.pkl' berhasil disimpan.

Proses 'Build' (penyimpanan artefak model) selesai.
File-file berikut telah dibuat di direktori kerja Anda:
- df_places.pkl
- tfidf_vectorizer.pkl
- cosine_sim.npy
- indices_map.pkl


In [ ]:
print("\n" + "="*80)
print("             CONTOH PENGGUNAAN MODEL REKOMENDASI             ")
print("="*80 + "\n")

# --- Contoh Rekomendasi Berbasis Konten ---
print("--- Rekomendasi Berbasis Konten ---")

target_place_1 = 'Pantai Natsepa'
print(f"\nMencari rekomendasi untuk '{target_place_1}'...")
recs_1 = get_content_based_recommendations(target_place_1, num_recommendations=5)
if not recs_1.empty:
    print(f"\nRekomendasi tempat wisata yang mirip dengan '{target_place_1}':")
    print(recs_1[['name', 'rating', 'reviews', 'review_keywords', 'description']].to_string())
else:
    print(f"Tidak ada rekomendasi yang ditemukan untuk '{target_place_1}'.")

print("\n" + "-"*50 + "\n")

target_place_2 = 'Pantai Ngurbloat Desa Ngilngof - Kei Islands'
print(f"\nMencari rekomendasi untuk '{target_place_2}'...")
recs_2 = get_content_based_recommendations(target_place_2, num_recommendations=5)
if not recs_2.empty:
    print(f"\nRekomendasi tempat wisata yang mirip dengan '{target_place_2}':")
    print(recs_2[['name', 'rating', 'reviews', 'review_keywords', 'description']].to_string())
else:
    print(f"Tidak ada rekomendasi yang ditemukan untuk '{target_place_2}'.")

print("\n" + "-"*50 + "\n")

print("Mencoba mencari tempat yang tidak ada atau salah ketik:")
recs_invalid = get_content_based_recommendations('pantai liang')
if recs_invalid.empty:
    print("Percobaan pencarian dengan nama yang salah ketik selesai.")


print("\n" + "="*80 + "\n")

# --- Contoh Rekomendasi Berdasarkan Popularitas ---
print("--- Rekomendasi Berdasarkan Popularitas (Rating & Ulasan) ---")

print("\nMencari 10 tempat wisata paling populer di dataset...")
popular_recs_df = get_popular_recommendations(df, num_recommendations=10)
if not popular_recs_df.empty:
    print("\nTop 10 Tempat Wisata Paling Populer (Berdasarkan Rating Tertinggi dan Jumlah Ulasan):")
    print(popular_recs_df[['name', 'rating', 'reviews', 'address']].to_string())
else:
    print("Tidak ada rekomendasi populer yang ditemukan.")

print("\n" + "="*80)
print("Seluruh proses pembuatan dan pengujian model rekomendasi selesai!")
print("="*80 + "\n")


             CONTOH PENGGUNAAN MODEL REKOMENDASI             

--- Rekomendasi Berbasis Konten ---

Mencari rekomendasi untuk 'Pantai Natsepa'...
Peringatan: Ditemukan duplikasi nama 'Pantai Natsepa'. Menggunakan indeks pertama: 0

Rekomendasi tempat wisata yang mirip dengan 'Pantai Natsepa':
                    name  rating  reviews                                                                                                review_keywords                                                                                                    description
197       Pantai Natsepa     4.4     2217  rujak, pemandangan, buah, es kelapa muda, banana boat, pisang goreng, bumbu kacang, gula merah, sukun, angkot  Jalur pasir putih dengan pemandian, banana boat & kayak, serta kafe "warung" standar yang menjual salad buah.
209       Pantai Natsepa     4.4     2217  rujak, pemandangan, buah, es kelapa muda, banana boat, pisang goreng, bumbu kacang, gula merah, sukun, angkot  Jalur pasir putih denga

In [ ]:
# Sel 5: Melakukan Inference (Pengujian Model & Penilaian Kualitatif)

print("--- Memulai Sesi Inference ---")
print("Anda bisa mencoba berbagai nama tempat atau jumlah rekomendasi di sini.")
print("="*70)

# --- INFERENCE UNTUK REKOMENDASI BERBASIS KONTEN ---
print("\n[Inferensi Model Rekomendasi Berbasis Konten]")

# 1. Tentukan nama tempat yang ingin dicari rekomendasinya
input_place_name = 'Pantai Ngurbloat Desa Ngilngof - Kei Islands' # Ganti dengan nama tempat lain
num_recommendations_content = 5 # Berapa banyak rekomendasi yang diinginkan

print(f"\nMencari {num_recommendations_content} rekomendasi yang mirip dengan '{input_place_name}'...")

# Dapatkan informasi tempat referensi
# Pastikan ini hanya mengambil satu baris jika ada nama duplikat
try:
    ref_idx = indices[input_place_name]
    if isinstance(ref_idx, pd.Series):
        ref_idx = ref_idx.iloc[0]
    reference_place_info = df.iloc[ref_idx]
    print(f"\n--- Informasi Tempat Referensi ('{input_place_name}') ---")
    print(f"Rating: {reference_place_info['rating']}")
    print(f"Reviews: {reference_place_info['reviews']}")
    print(f"Keywords: {reference_place_info['review_keywords']}")
    print(f"Description (potongan): {reference_place_info['description'][:150]}...")
    print("-" * 40)
except KeyError:
    print(f"Tempat referensi '{input_place_name}' tidak ditemukan di dataset.")
    reference_place_info = None


# Panggil fungsi rekomendasi berbasis konten
recommended_by_content = get_content_based_recommendations(
    place_name=input_place_name,
    num_recommendations=num_recommendations_content
)

# Tampilkan hasilnya
if not recommended_by_content.empty:
    print(f"\nHasil Rekomendasi Berbasis Konten untuk '{input_place_name}':")
    print("--- Perhatikan 'review_keywords' dan 'description' untuk kemiripan konten ---")
    # Tampilkan kolom yang relevan
    print(recommended_by_content[['name', 'rating', 'reviews', 'review_keywords', 'description']].to_string())

    # Contoh "parameter penilaian" kualitatif:
    print("\n--- Penilaian Kualitatif (Inspeksi Manual) ---")
    print("Bandingkan 'review_keywords' dan 'description' tempat referensi dengan rekomendasi.")
    print("Apakah tempat-tempat ini memiliki karakteristik, aktivitas, atau jenis pemandangan yang serupa?")
    print("Contoh: Jika 'Pantai Natsepa' punya keyword 'rujak', apakah rekomendasi juga terkait makanan/pantai serupa?")
else:
    print(f"Tidak ada rekomendasi konten yang ditemukan untuk '{input_place_name}'. Pastikan nama tempat benar.")

print("\n" + "-"*70)

# 2. Coba input nama tempat yang lain
input_place_name_2 = 'Pantai Ora'
num_recommendations_content_2 = 3

print(f"\nMencari {num_recommendations_content_2} rekomendasi yang mirip dengan '{input_place_name_2}'...")
try:
    ref_idx_2 = indices[input_place_name_2]
    if isinstance(ref_idx_2, pd.Series):
        ref_idx_2 = ref_idx_2.iloc[0]
    reference_place_info_2 = df.iloc[ref_idx_2]
    print(f"\n--- Informasi Tempat Referensi ('{input_place_name_2}') ---")
    print(f"Rating: {reference_place_info_2['rating']}")
    print(f"Reviews: {reference_place_info_2['reviews']}")
    print(f"Keywords: {reference_place_info_2['review_keywords']}")
    print(f"Description (potongan): {reference_place_info_2['description'][:150]}...")
    print("-" * 40)
except KeyError:
    print(f"Tempat referensi '{input_place_name_2}' tidak ditemukan di dataset.")
    reference_place_info_2 = None


recommended_by_content_2 = get_content_based_recommendations(
    place_name=input_place_name_2,
    num_recommendations=num_recommendations_content_2
)

if not recommended_by_content_2.empty:
    print(f"\nHasil Rekomendasi Berbasis Konten untuk '{input_place_name_2}':")
    print("--- Perhatikan 'review_keywords' dan 'description' untuk kemiripan konten ---")
    print(recommended_by_content_2[['name', 'rating', 'reviews', 'review_keywords', 'description']].to_string())
else:
    print(f"Tidak ada rekomendasi konten yang ditemukan untuk '{input_place_name_2}'. Pastikan nama tempat benar.")

print("\n" + "="*70)

# --- INFERENCE UNTUK REKOMENDASI BERDASARKAN POPULARITAS ---
print("\n[Inferensi Model Rekomendasi Berdasarkan Popularitas]")

num_recommendations_popular = 7 # Berapa banyak tempat populer yang diinginkan

print(f"\nMencari {num_recommendations_popular} tempat wisata paling populer di dataset...")

# Panggil fungsi rekomendasi popularitas
popular_recommendations = get_popular_recommendations(num_recommendations=num_recommendations_popular)

# Tampilkan hasilnya
if not popular_recommendations.empty:
    print(f"\nHasil Rekomendasi Berdasarkan Popularitas (Top {num_recommendations_popular}):")
    print("--- Perhatikan 'rating' dan 'reviews' untuk popularitas ---")
    print(popular_recommendations[['name', 'rating', 'reviews', 'address']].to_string())
    print("\n--- Penilaian Kualitatif (Inspeksi Manual) ---")
    print("Pastikan daftar ini diurutkan dari rating tertinggi ke terendah,")
    print("dan untuk rating yang sama, diurutkan dari jumlah ulasan terbanyak.")
else:
    print("Tidak ada rekomendasi populer yang ditemukan.")

print("\n" + "="*70)
print("Sesi Inference Selesai.")
print("Perlu diingat: 'Akurasi' dalam arti tradisional tidak berlaku untuk model rekomendasi ini.")
print("Penilaian dilakukan secara kualitatif, melihat relevansi dan logisnya rekomendasi.")

--- Memulai Sesi Inference ---
Anda bisa mencoba berbagai nama tempat atau jumlah rekomendasi di sini.

[Inferensi Model Rekomendasi Berbasis Konten]

Mencari 5 rekomendasi yang mirip dengan 'Pantai Ngurbloat Desa Ngilngof - Kei Islands'...

--- Informasi Tempat Referensi ('Pantai Ngurbloat Desa Ngilngof - Kei Islands') ---
Rating: 4.7
Reviews: 1070
Keywords: pasir, sunset, panjang, tepung, pemandangan, surga, cottage, voli, masyarakat, pisang
Description (potongan): Hamparan pasir putih panjang berlatar pohon kelapa, populer untuk melihat sunset dan bermain voli....
----------------------------------------
Peringatan: Ditemukan duplikasi nama 'Pantai Ngurbloat Desa Ngilngof - Kei Islands'. Menggunakan indeks pertama: 2

Hasil Rekomendasi Berbasis Konten untuk 'Pantai Ngurbloat Desa Ngilngof - Kei Islands':
--- Perhatikan 'review_keywords' dan 'description' untuk kemiripan konten ---
                                             name  rating  reviews                                     